In [294]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [295]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [296]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [297]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [298]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [299]:
df.rename(columns=({'Survived': 'y'}), inplace=True)

In [300]:
def prepare_df(a):
    df = a.copy()
    df = df.drop(['Name', 'Ticket', 'Embarked'], axis=1)
    df['Sex'] = df['Sex'].map({'male': 1, 'female': 0})
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Cabin'] = df['Cabin'].fillna(0)
    df['Cabin'] = df['Cabin'].apply(lambda n: 1 if n != 0 else 0).astype(int)
    return df

In [301]:
df = prepare_df(df)
test = prepare_df(test)


In [302]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
df[['Age', 'Fare']] = scaler.fit_transform(df[['Age', 'Fare']])
test[['Age', 'Fare']] = scaler.fit_transform(test[['Age', 'Fare']])

In [303]:
X = df.drop('y', axis=1)
y = df['y']

In [304]:
df.isnull().sum()

PassengerId    0
y              0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Cabin          0
dtype: int64

In [305]:
df.head()

,PassengerId,y,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin
0,1,0,3,1,-0.592481,1,0,-0.502445,0
1,2,1,1,0,0.638789,1,0,0.786845,1
2,3,1,3,0,-0.284663,0,0,-0.488854,0
3,4,1,1,0,0.407926,1,0,0.420730,1
4,5,0,3,1,0.407926,0,0,-0.486337,0


In [306]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=101, shuffle=True)

In [307]:
n_estimators = [100, 150, 200]
criterion = ['entropy', 'gini']
max_depth = [2, 5, 7]
params_grid = {'n_estimators': n_estimators,
               'criterion': criterion,
               'max_depth': max_depth}

In [308]:
grid = GridSearchCV(RandomForestClassifier(n_jobs=-1), params_grid, n_jobs=-1, verbose=True,cv=5)

In [309]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=-1), n_jobs=-1,
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': [2, 5, 7],
                         'n_estimators': [100, 150, 200]},
             verbose=True)

In [310]:
grid.best_params_

{'criterion': 'entropy', 'max_depth': 7, 'n_estimators': 200}

In [311]:
y_pred = grid.predict(X_test)

In [312]:
from sklearn.metrics import roc_auc_score


roc_auc_score(y_test, y_pred)

0.8259469696969697

In [313]:
output = pd.DataFrame({'PassengerId': X_test['PassengerId'], 'Survived': y_pred})
output.to_csv('submission.csv', index=False)
